# LIBRARIES

In [192]:
from csv import reader 
import pandas as pd 
import numpy as np
from sklearn.datasets import make_classification 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

# EXPORTING DATA

In [193]:
def imfile(name) : 
    file = pd.read_csv(r'C:\Users\macie\Documents\BPL\{}.csv'.format(name))
    file = file[['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'HY', 'AY', 'HR','AR','B365H','B365A','B365D']]
    return file

In [194]:
def flat(array) : 
    l = len(array)
    new = np.zeros((l,1))
    i = 0
    for element in array : 
        new[i,0] = element
        i = i + 1
    return new

In [195]:
files = ['I18','I17','I16','I15','I14','I13','I12','I11','I10','I9','I8','I7','I6','I5','B19','B18','B17','B16','B15','B14','B13','B12','B11','B10','B9','B8','B7','B6','S19','S18','S17','S16','S15','S14','S13','S12','S11','S10','S9','S8','S7','S6','S5','E19','E18','E17','E16','E15','E14','E13','E12','E11','E10','E9','E8','E7','E6','E5','F19','F18','F17','F16','F15','F14','F13','F12','F11','F10','F9','F8','F7','F6','F5']
ALL = imfile('I19')
for element in files : 
    file = imfile(element)
    ALL = pd.concat([file,ALL])
ALL = ALL[::-1] # reverse the table 
ALL = np.array((ALL))
[R,C] = ALL.shape
help_column = []

In [196]:
i = 0
for i in range(R) : 
    i = i + 1
    help_column.append(i)
help_columns = flat(help_column)
help_columns.shape

(26583, 1)

# CREATING FORM FUNCTION WHICH ANALYZE THE FORM FROM LAST 7 MATCHES FOR EACH TEAM

In [197]:
def form(team,start) :
    ph = 0
    x = 0
    for row in ALL[start:] :
        if x == 7 : 
            break
        if row[0] == team :
            x = x + 1
            if row[4] == 'H' : 
                ph = ph + 3
            if row[4] == 'D' :
                ph = ph + 1
            if row[4] == 'A' :
                ph = ph
        elif row[1] == team :
            x = x + 1
            if row[4] == 'A' :
                ph = ph + 3
            if row[4] == 'D' : 
                ph = ph + 1
            if row[4] == 'H' : 
                ph = ph
    return ph/21

# AVERAGE ACCURACY FROM LAST 10 MATCHES

In [198]:
def accuracy(team, start) :
    accuracy = 0
    counter = 0
    for row in ALL[start:] :
        if counter == 10 :
            break
        if row[0] == team : 
            counter = counter + 1
            shots = row[5]
            ontarget = row[7]
            if ontarget == 0 : 
                acc = 0 
            else :
                acc = shots / ontarget
            accuracy = accuracy + acc
        elif row[1] == team : 
            counter = counter + 1 
            shots = row[6]
            ontarget = row[8]
            if ontarget == 0 : 
                acc = 0 
            else :
                acc = shots / ontarget
            accuracy = accuracy + acc
    return accuracy

# HOW MANY DRAWS, WINS, LOSES HAD TEAM IN LAST 10 MATCHES

In [199]:
def score(team, start, result) :
    counter = 0
    results = 0
    for row in ALL[start:] :
        if counter == 10 :
            break
        if row[0] == team :
            counter = counter + 1
            if row[4] == result :
                results = results + 1
        elif row[1] == team : 
            counter = counter + 1 
            if row[4] == result : 
                results = results + 1
    return results
                 

# CARDS THAT TEAM IS SCORING AVERAGE IN LAST 15 MATCHES

In [200]:
def yellows(team, start) : 
    counter = 0
    cards = 0
    for row in ALL[start:] : 
        if counter == 15 : 
            break 
        if row[0] == team : 
            counter = counter + 1
            cards = cards + row[9] 
        elif row[1] == team : 
            counter = counter + 1 
            cards = cards + row[10]
    return cards / 15 
def reds(team, start) : 
    counter = 0
    cards = 0
    for row in ALL[start:] : 
        if counter == 15 :
            break 
        if row[0] == team : 
            counter = counter + 1
            cards = cards + row[11] 
        elif row[1] == team : 
            counter = counter + 1 
            cards = cards + row[12]
    return cards / 15

# AVERAGE NUMBER OF GOALS FROM LAST 15 MATCHES

In [201]:
def goals(team, start) : 
    counter = 0 
    goals = 0 
    for row in ALL[start:] :
        if counter == 15 : 
            break
        if row[0] == team : 
            counter = counter + 1
            goals = goals + row[2]
        if row[1] == team : 
            counter = counter + 1
            goals = goals + row[3]
    return goals / 15

# CREATING FEATURES

In [202]:
hf = [] # home form
ha = [] # home accuracy
hw = [] # home wins
hd = [] # home draws
hl = [] # home loses
hy = [] # home yellow cards
hr = [] # home red cards
hg = [] # home goals
af = [] # same as above but for away team
aa = [] 
aw = []
ad = [] 
al = []
ay = []
ar = []
ag = []
HOME = [ha,hw,hd,hl,hy,hr,hg]
AWAY = [af,aa,aw,ad,al,ay,ar,ag]

In [203]:
p = 0 # counter for a position in the for loop 
for row in ALL : 
    h = row[0] # name of home team
    a = row[1] # name of away team
    hf.append(form(h,p))
    ha.append(accuracy(h,p))
    hw.append(score(h,p,'H'))
    hd.append(score(h,p,'D'))
    hl.append(score(h,p,'A'))
    hy.append(yellows(h,p))
    hr.append(reds(h,p))
    hg.append(goals(h,p))
    p = p + 1

In [204]:
p = 0 
for row in ALL : 
    h = row[0]
    a = row[1]
    af.append(form(a,p))
    aa.append(accuracy(a,p))
    aw.append(score(a,p,'H'))
    ad.append(score(a,p,'D'))
    al.append(score(a,p,'A'))
    ay.append(yellows(a,p))
    ar.append(reds(a,p))
    ag.append(goals(a,p))
    p = p + 1

# TURN FEATURES INTO NUMPY AND STACK THEM TOGETHER


In [205]:
feature = flat(hf)
for element in HOME : 
    new = flat(element)
    feature = np.hstack((feature,new))
for element in AWAY : 
    new = flat(element)
    feature = np.hstack((feature,new))

# PREPARING DATA FOR MACHINE LOGISTIC REGRESSION

In [217]:
# 1 ODDS HOME, ODDS DRAW, ODDS AWAY, HOME FORM, HOME ACCURACY, HOME WINS,DRAWS,LOSES, HOME YELLOWS, HOME REDS
ODDS = ALL[:,[13,15,14,4]]
X = np.hstack((ODDS,feature))
X = np.hstack((X, help_columns))
X = pd.DataFrame((X))
X = X.dropna()
y = X[3]
num = X[20]
X = X.drop(3,axis = 1)
X = X.drop(20, axis = 1)

In [218]:
X.to_csv(r'C:\Users\macie\Documents\X.csv', index = True)
y.to_csv(r'C:\Users\macie\Documents\y.csv', index = True)
num.to_csv(r'C:\Users\macie\Documents\num.csv', index = True)

In [207]:
X = np.array((X))
[r,c] = X.shape
for i in range(c) :
    X[:,i] = preprocessing.scale(X[:,i])
X = X.astype(float)

In [208]:
num = num - 1
num = np.asarray(num)

# SPLITTING THE DATA

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70,test_size=0.30, random_state=101)

# PERFORMING LOGISTIC REGRESSION

In [210]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [211]:
y_pred = log_reg.predict(X_test)

# THE ACCURACY OF THE MODEL

In [212]:
score = log_reg.score(X_test, y_test)
score

0.7109088610795096

# CHECKING THE RESULTS ON A 10 TEST MATCHES

In [213]:
r,c = X_train.shape
y_test = np.array((y_test))
j = 0

In [220]:
for i in range(10) : 
    j = j + 1
    position = int(num[r+j])
    home = ALL[position, 0]
    away = ALL[position, 1]
    pred = y_pred[j]
    res = y_test[j]
    if pred == res : 
        info = 'CORRECT'
    else : 
        info = 'INCORRECT'
    print('Match : {} - {}, Prediction : {} Actual result : {} {}'.format(home,away,pred,res,info))

Match : Man United - Stoke, Prediction : H Actual result : H CORRECT
Match : Everton - Man City, Prediction : A Actual result : A CORRECT
Match : Man City - Tottenham, Prediction : A Actual result : A CORRECT
Match : Arsenal - Man United, Prediction : A Actual result : H INCORRECT
Match : Wolves - Aston Villa, Prediction : A Actual result : A CORRECT
Match : Sunderland - Swansea, Prediction : H Actual result : H CORRECT
Match : Stoke - West Brom, Prediction : A Actual result : D INCORRECT
Match : QPR - Wigan, Prediction : H Actual result : H CORRECT
Match : Norwich - Chelsea, Prediction : H Actual result : D INCORRECT
Match : Fulham - Newcastle, Prediction : D Actual result : D CORRECT
